<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/NetGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import torch.optim as optim
import time
from math import sqrt
import matplotlib.pyplot as plt

# select CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if str(device) == 'cuda:0':
	print('CUDA device selected!')
elif str(device) == 'cpu':
	print('CUDA device not available. CPU selected')

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, 462*5)
        self.fc2 = nn.Linear(462*5, 462*5)
        self.fc3 = nn.Linear(462*5, 462*10)
        self.fc4 = nn.Linear(462*10, 231*461)
        self.dropout = nn.Dropout(0.1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
       	x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

net = MyNet().to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-4)

loss_v = []

for n in range(1,8):
    d_PATH = f'drive/MyDrive/YashData/{n}'
    f = open(d_PATH, 'rb')
    data = pickle.load(f)

    for i in range(1000):
        optimizer.zero_grad()

        input = torch.from_numpy(data[i][1]).float().to(device)
        true_y = torch.FloatTensor(data[i][0]).to(device)

        output = net(input).to(device)

        loss = criterion(output, true_y).to(device)
        loss_v.append(sqrt(loss.item()))
        loss.backward()

        optimizer.step()

        print(f'File {n}: Epoch {i}: {sqrt(loss.item())}')

PATH = 'model.pth'
torch.save(net.state_dict(), PATH)

plt.plot(loss_v[70:])
plt.savefig('Loss.png')
plt.show()


CUDA device selected!
File 1: Epoch 0: 0.576910754218913
File 1: Epoch 1: 0.5775029470112567
File 1: Epoch 2: 0.5741545972215324
File 1: Epoch 3: 0.5705117961740543
File 1: Epoch 4: 0.5676094167318015
File 1: Epoch 5: 0.564571302245868
File 1: Epoch 6: 0.5585963375858582
File 1: Epoch 7: 0.5516905887128782
File 1: Epoch 8: 0.5438302978331927
File 1: Epoch 9: 0.5357349914000736
File 1: Epoch 10: 0.5225723860761015
File 1: Epoch 11: 0.5107976762960718
File 1: Epoch 12: 0.4945523503097363
File 1: Epoch 13: 0.47478525803801963
File 1: Epoch 14: 0.4535816949496704
File 1: Epoch 15: 0.4297161092770666
File 1: Epoch 16: 0.4028803755910204
File 1: Epoch 17: 0.378016222921039
File 1: Epoch 18: 0.34564384124543857
File 1: Epoch 19: 0.31653202759395554
File 1: Epoch 20: 0.3023098584087894
File 1: Epoch 21: 0.29522553026124426
File 1: Epoch 22: 0.3022864196202782
File 1: Epoch 23: 0.3226744515039658
File 1: Epoch 24: 0.3313760238812953
File 1: Epoch 25: 0.3439031064913708
File 1: Epoch 26: 0.34237